In [1]:
import time
from core.utils import Tibanna
from dcicutils import ff_utils
from core.wfr import *

wf_md5 = "md5"

env = 'data'
tibanna = Tibanna(env=env)
exclude_miseq = True
run_md5 = raw_input("Do you wanna run md5 if missing? (md5/none)")


def step_settings(seq, my_organism=""):
    genome = ""
    mapper = {'human':'GRCh38','mouse':'GRCm38'}
    genome = mapper.get(my_organism)
    
    wf_dict =[{
        'wf_name': 'md5',
        'wf_uuid': 'd3f25cd3-e726-4b3c-a022-48f844474b41',
        'parameters': {}
        }]
    return wf_dict[seq]



raw_bucket = tibanna.s3.raw_file_bucket
out_bucket = tibanna.s3.outfile_bucket

file_url = '/search/?status=uploading&status=upload%20failed&type=FileFastq'
# file_url = '/search/?type=FileReference'
# file_url = '/search/?status=uploading&status=upload%20failed&type=FileProcessed'
all_files = ff_utils.search_metadata(file_url, ff_env=env)
print len(all_files)

printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            print "Deleted File", a_file
            continue
    except:
        print a_file
        break
        
    if counter-printn > 100:
        print counter
        printn = counter

    if 'FileProcessed' in a_file['@type']:
            my_bucket = out_bucket
    else:  # covers cases of FileFastq, FileReference, FileMicroscopy
            my_bucket = raw_bucket       
 
    # check if file is in s3
    head_info = tibanna.s3.does_key_exist(a_file['upload_key'], my_bucket)
    if not head_info:
        print a_file['accession'], "does not have a file in S3"
        continue
        
    md5 = False
    file_id = a_file['accession']
    status = a_file.get('status')
    workflows = a_file.get('workflow_run_inputs')

    if a_file.get('content_md5sum'):
        md5 = True
        
    # is there md5sum for gzip
    if not a_file.get('md5sum'):
        print file_id,"does not have the md5sum calculated during upload"
    # Check workflows f
    md5_status = 'did_not_run'
    
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    md5_status = get_wfr_out(a_file['uuid'], "md5 0.2.5", None, env)
    if md5_status not in ['complete', 'running']:
        md5_status = get_wfr_out(a_file['uuid'], "md5", None, env)
    
    if md5_status == 'running':
        print(file_id, 'still running')
        continue 

    if status in ["uploading", "upload failed"] and md5_status != 'complete':
        # if not, shall we run it?
        if run_md5 == 'md5':
            print 'md5 running for', file_id
            inp_f = {'input_file':file_id}
            run_missing_wfr(step_settings(0), inp_f, file_id, env, tibanna)
            print ''
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_id
            
    
    elif status in ["uploading", "upload failed"] and md5_status == 'complete':
        print "ERROR:",file_id ,"There is a successful md5 run but status not switched"
        
#         # if not, shall we run it?
#         if run_md5 == 'md5':
#             print 'md5 running for', file_id
#             inp_f = {'input_file':file_id}
#             run_missing_wfr(step_settings(0), inp_f, file_id, env, tibanna)
#             print ''
#             time.sleep(30)
#         # user does not want it to be run, so just report
#         else:
#             print 'md5 run missing for', file_id
        
    elif md5_status != 'complete':
        print file_id, status, 'file does not have md5 run'
#         # if not, shall we run it?
#         if run_md5 == 'md5':
#             print 'md5 running for', file_id
#             code_md5= "invoke run_md5 " + env + " " + file_resp['display_title'] + " " + file_resp['uuid']
#             run(code_md5)
#             print ''
#             time.sleep(30)
#         # user does not want it to be run, so just report
#         else:
#             print 'md5 run missing for', file_id
            
    print



Do you wanna run md5 if missing? (md5/none)
7
object fa07e132-e8c7-4251-947c-c1efb9e1afb1/4DNFIP389H26.fastq.gz not found on bucket elasticbeanstalk-fourfront-webprod-files
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIP389H26 does not have a file in S3
object abddc01d-a53d-4779-9d29-d0bfa7ce47e0/4DNFIRZINV8H.fastq.gz not found on bucket elasticbeanstalk-fourfront-webprod-files
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIRZINV8H does not have a file in S3
object d06c7918-7a46-45c7-a0e3-f0b32665f404/4DNFIH77OSA2.fastq.gz not found on bucket elasticbeanstalk-fourfront-webprod-files
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIH77OSA2 does not have a file in S3
object 90186fa5-9eab-4713-92a7-bedbb8621c57/4DNFILF1UY2Y.fastq.gz not found on bucket elasticbeanstalk-fourfront-webprod-files
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFILF1UY2Y does not have a file 